In [1]:
# Load the SQL extension for Jupyter Notebook
%load_ext sql

# Imports for pandas, numpy, pyodbc, and sqlalchemy
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine, Date, Integer, Float, String, BigInteger, DateTime
from urllib.parse import quote_plus

In [2]:
# Set up the connection string
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=COVID;'
    'Trusted_Connection=yes;'
)
# Format the connection string for SQLAlchemy and Jupyter magic
connection_uri = f"mssql+pyodbc:///?odbc_connect={quote_plus(conn_str)}"
%sql $connection_uri


In [ ]:
# Load the Excel files into DataFrames
df_deaths = pd.read_excel("D:/Google Drive/Sheets/Data Ana/covid_deaths.xlsx")
df_vacs = pd.read_excel("D:/Google Drive/Sheets/Data Ana/covid_Vacs.xlsx")

In [ ]:
# Data type dictionaries
dtype_dict_deaths = {
    'iso_code': 'VARCHAR(255)', 'continent': 'VARCHAR(255)', 'location': 'VARCHAR(255)', 
    'date': 'DATETIME', 'population': 'BIGINT', 'total_cases': 'FLOAT', 'new_cases': 'FLOAT', 
    'new_cases_smoothed': 'FLOAT', 'total_deaths': 'FLOAT', 'new_deaths': 'FLOAT', 
    'new_deaths_smoothed': 'FLOAT', 'total_cases_per_million': 'FLOAT', 'new_cases_per_million': 'FLOAT', 
    'new_cases_smoothed_per_million': 'FLOAT', 'total_deaths_per_million': 'FLOAT', 
    'new_deaths_per_million': 'FLOAT', 'new_deaths_smoothed_per_million': 'FLOAT', 
    'reproduction_rate': 'FLOAT', 'icu_patients': 'FLOAT', 'icu_patients_per_million': 'FLOAT', 
    'hosp_patients': 'FLOAT', 'hosp_patients_per_million': 'FLOAT', 'weekly_icu_admissions': 'FLOAT', 
    'weekly_icu_admissions_per_million': 'FLOAT', 'weekly_hosp_admissions': 'FLOAT', 
    'weekly_hosp_admissions_per_million': 'FLOAT'
}

In [ ]:
# Data type dictionaries


dtype_dict_vacs = {
    'iso_code': 'VARCHAR(255)', 'continent': 'VARCHAR(255)', 'location': 'VARCHAR(255)', 
    'date': 'DATETIME', 'total_tests': 'FLOAT', 'new_tests': 'FLOAT', 'total_tests_per_thousand': 'FLOAT', 
    'new_tests_per_thousand': 'FLOAT', 'new_tests_smoothed': 'FLOAT', 
    'new_tests_smoothed_per_thousand': 'FLOAT', 'positive_rate': 'FLOAT', 'tests_per_case': 'FLOAT', 
    'tests_units': 'VARCHAR(255)', 'total_vaccinations': 'FLOAT', 'people_vaccinated': 'FLOAT', 
    'people_fully_vaccinated': 'FLOAT', 'total_boosters': 'FLOAT', 'new_vaccinations': 'FLOAT', 
    'new_vaccinations_smoothed': 'FLOAT', 'total_vaccinations_per_hundred': 'FLOAT', 
    'people_vaccinated_per_hundred': 'FLOAT', 'people_fully_vaccinated_per_hundred': 'FLOAT', 
    'total_boosters_per_hundred': 'FLOAT', 'new_vaccinations_smoothed_per_million': 'FLOAT', 
    'new_people_vaccinated_smoothed': 'FLOAT', 'new_people_vaccinated_smoothed_per_hundred': 'FLOAT', 
    'stringency_index': 'FLOAT', 'population_density': 'FLOAT', 'median_age': 'FLOAT', 
    'aged_65_older': 'FLOAT', 'aged_70_older': 'FLOAT', 'gdp_per_capita': 'FLOAT', 
    'extreme_poverty': 'FLOAT', 'cardiovasc_death_rate': 'FLOAT', 'diabetes_prevalence': 'FLOAT', 
    'female_smokers': 'FLOAT', 'male_smokers': 'FLOAT', 'handwashing_facilities': 'FLOAT', 
    'hospital_beds_per_thousand': 'FLOAT', 'life_expectancy': 'FLOAT', 'human_development_index': 'FLOAT', 
    'excess_mortality_cumulative_absolute': 'FLOAT', 'excess_mortality_cumulative': 'FLOAT', 
    'excess_mortality': 'FLOAT', 'excess_mortality_cumulative_per_million': 'FLOAT'
}

In [ ]:
# Round float columns to 2 decimal places
float_columns_deaths = [col for col, dtype in dtype_dict_deaths.items() if dtype == 'FLOAT']
float_columns_vacs = [col for col, dtype in dtype_dict_vacs.items() if dtype == 'FLOAT']

df_deaths[float_columns_deaths] = df_deaths[float_columns_deaths].round(2)
df_vacs[float_columns_vacs] = df_vacs[float_columns_vacs].round(2)

# Define the function to insert DataFrame in smaller chunks with SQLAlchemy type mapping
def insert_dataframe_in_chunks(df, table_name, engine, dtype_dict, chunk_size=1000):
    # Create SQLAlchemy type mapping
    dtype_mapping = {col: (BigInteger() if dtype == 'BIGINT' else
                           Float() if dtype == 'FLOAT' else
                           DateTime() if dtype == 'DATETIME' else
                           String(length=255) if dtype == 'VARCHAR(255)' else
                           Integer())
                     for col, dtype in dtype_dict.items()}
    
    for start in range(0, len(df), chunk_size):
        end = start + chunk_size
        try:
            df[start:end].to_sql(table_name, engine, if_exists='append', index=False, dtype=dtype_mapping)
        except Exception as e:
            print(f"Error inserting chunk from {start} to {end}: {e}")

# Create a SQLAlchemy engine
engine = create_engine(connection_uri)

# Insert DataFrames into the database in smaller chunks
insert_dataframe_in_chunks(df_deaths, "covid_deaths", engine, dtype_dict_deaths)
insert_dataframe_in_chunks(df_vacs, "covid_vaccinations", engine, dtype_dict_vacs)

In [ ]:
%%sql
SELECT * FROM covid_deaths
ORDER BY 1,2 -- Placeholder for ORDER BY clause
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY

In [ ]:
%%sql

-- looking at the total cases vs total deaths


Select location, total_cases, new_cases, total_deaths, population
From covid_deaths
Order by 1
OFFSET 0 ROWS FETCH NEXT 300 ROWS ONLY


In [ ]:
%%sql
-- looking at the total cases vs total deaths in US, showing death percentage

Select location, date, total_cases,total_deaths,(total_deaths/total_cases)*100 as DeathPercentage
from covid_deaths
where location like '%States%'
order by 1,2 
OFFSET 0 ROWS FETCH NEXT 300 ROWS ONLY

In [ ]:
%%sql
-- looking at the total cases vs population in US, showing infection percentage

Select location, "date", total_cases,population,(total_cases/population)*100 as CasePercentage
from covid_deaths
where location like '%States%'
order by 1,2 


In [ ]:
%%sql
-- showing countries with highest infection rate compared to population

SELECT 
    location, 
    MAX(total_cases) AS highest_infection_count,
    population,
    MAX((total_cases/population))*100 AS infection_percentage
FROM 
    covid_deaths
GROUP BY 
    location, 
    population
ORDER BY
    infection_percentage DESC
OFFSET 0 ROWS FETCH NEXT 100 ROWS ONLY

In [ ]:
%%sql
--show  countries with highest death count per location
SELECT 
    location, max(total_deaths) as total_death_count
FROM  covid_deaths
WHERE continent IS not NULL
GROUP BY location
ORDER BY total_death_count DESC
--OFFSET 0 ROWS FETCH NEXT 100 ROWS ONLY


In [ ]:
%%sql
--show  continent with highest death count 
SELECT 
    continent, max(total_deaths) as total_death_count
FROM  covid_deaths
WHERE continent IS  not NULL
GROUP BY continent
ORDER BY total_death_count DESC
OFFSET 0 ROWS FETCH NEXT 100 ROWS ONLY

In [ ]:
%%sql
select "date",sum(new_deaths) as total_new_deaths from covid_deaths 
where "date" > '2024-01-01'
and total_deaths is not null
group by "date"
order by "date"

In [ ]:
%%sql

SELECT sum(new_cases) as total_new_cases, sum(new_deaths) as total_new_deaths, sum(new_deaths)/sum(new_cases)*100 as DeathPercentage
FROM  covid_deaths
--WHERE continent IS  NULL



In [ ]:
%%sql

-- Query to fetch data from covid_deaths and covid_vaccinations
SELECT 
    deaths.*, 
    vacs.*
FROM 
    covid_deaths AS deaths
JOIN 
    covid_vaccinations AS vacs
ON 
    deaths.location = vacs.location
AND 
    deaths."date" = vacs."date"
WHERE 
    deaths.continent IS NOT NULL
ORDER BY 
    deaths.location, 
    deaths."date"
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY;

In [ ]:
%%sql
-- Looking at total population vs vaccination
CREATE INDEX idx_deaths_location_date ON covid_deaths(location, "date");
CREATE INDEX idx_vacs_location_date ON covid_vaccinations(location, "date");

 SELECT 
        deaths.continent, 
        deaths.location, 
        deaths."date", 
        deaths.population, 
        vacs.new_vaccinations,
        SUM(vacs.new_vaccinations) OVER (PARTITION BY deaths.location ORDER BY deaths."date") AS rolling_people_vaccinated
    FROM 
        covid_deaths AS deaths
    JOIN 
        covid_vaccinations AS vacs
    ON 
        deaths.location = vacs.location
    AND 
        deaths."date" = vacs."date"
    WHERE 
        deaths.continent IS NOT NULL
    AND 
        vacs.new_vaccinations IS NOT NULL
ORDER BY 2, 3
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY


In [ ]:
%%sql

-- Creating indexes to optimize the query
CREATE INDEX idx_deaths_location_date ON covid_deaths(location, "date");
CREATE INDEX idx_vacs_location_date ON covid_vaccinations(location, "date");

WITH pop_vs_vac (continent, location, "date", population, new_vaccinations, rolling_people_vaccinated) AS (
    SELECT 
        deaths.continent, 
        deaths.location, 
        deaths."date", 
        deaths.population, 
        vacs.new_vaccinations,
        SUM(vacs.new_vaccinations) OVER (PARTITION BY deaths.location ORDER BY deaths."date") AS rolling_people_vaccinated
    FROM 
        covid_deaths AS deaths
    JOIN 
        covid_vaccinations AS vacs
    ON 
        deaths.location = vacs.location
    AND 
        deaths."date" = vacs."date"
    WHERE 
        deaths.continent IS NOT NULL
    AND 
        vacs.new_vaccinations IS NOT NULL
)

SELECT 
    *, 
    (rolling_people_vaccinated / population) * 100 AS percent_population_vaccinated
FROM 
    pop_vs_vac
ORDER BY 
    location, "date"
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY;

In [ ]:
%%sql

CREATE TABLE percent_population_vaccinated
(
continent VARCHAR(24),
location VARCHAR(24),
"date" DATETIME,
population BIGINT,
new_vaccinations BIGINT,
rolling_people_vaccinated BIGINT
)



In [ ]:
%%sql

-- Ensure the table definition can hold the data lengths

DROP TABLE IF EXISTS percent_population_vaccinated
CREATE TABLE percent_population_vaccinated
(
    continent VARCHAR(255),
    location VARCHAR(255),
    "date" DATETIME,
    population BIGINT,
    new_vaccinations BIGINT,
    rolling_people_vaccinated BIGINT
);

-- Insert data into the table
INSERT INTO percent_population_vaccinated
SELECT 
    deaths.continent, 
    deaths.location, 
    deaths."date", 
    deaths.population, 
    vacs.new_vaccinations,
    SUM(vacs.new_vaccinations) OVER (PARTITION BY deaths.location ORDER BY deaths."date") AS rolling_people_vaccinated
FROM 
    covid_deaths AS deaths
JOIN 
    covid_vaccinations AS vacs
ON 
    deaths.location = vacs.location
AND 
    deaths."date" = vacs."date"
WHERE 
    deaths.continent IS NOT NULL
AND 
    vacs.new_vaccinations IS NOT NULL;

-- Select from the table and calculate the percentage
SELECT 
    *, 
    (rolling_people_vaccinated / population) * 100 AS percent_population_vaccinated
FROM 
    percent_population_vaccinated
ORDER BY 
    location, "date"



In [3]:
%%sql
SELECT * 
FROM 
    percent_population_vaccinated
ORDER BY 
    location, "date"
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY


 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3Dlocalhost%3BDATABASE%3DCOVID%3BTrusted_Connection%3Dyes%3B
Done.


continent,location,date,population,new_vaccinations,rolling_people_vaccinated
Asia,Afghanistan,2021-05-27 00:00:00,41128772,2859,2859
Asia,Afghanistan,2021-06-03 00:00:00,41128772,4015,6874
Asia,Afghanistan,2022-01-27 00:00:00,41128772,6868,13742
Asia,Afghanistan,2022-04-27 00:00:00,41128772,383,14125
Asia,Afghanistan,2022-09-12 00:00:00,41128772,9447,23572
Asia,Afghanistan,2022-11-02 00:00:00,41128772,36587,60159
Asia,Afghanistan,2022-11-16 00:00:00,41128772,14800,74959
Asia,Afghanistan,2023-04-25 00:00:00,41128772,3316,78275
Europe,Albania,2021-01-13 00:00:00,2842318,60,60
Europe,Albania,2021-01-14 00:00:00,2842318,78,138
